## LEGENDA 'Conservation Status'
Least Concern (LC): Indica che una specie non è attualmente a rischio di estinzione. Queste specie sono considerate relativamente sicure in natura.

Vulnerable (VU): Indica che una specie è a rischio di estinzione in natura, ma non è ancora considerata in pericolo critico. Le specie vulnerabili potrebbero avere una popolazione ridotta o affrontare minacce significative nell'ambiente circostante.

Endangered (EN): Indica che una specie è a rischio di estinzione imminente in natura. Queste specie hanno una popolazione molto bassa e/o affrontano minacce gravi che mettono a rischio la loro sopravvivenza.

Not Evaluated (NE): Indica che lo stato di conservazione di una specie non è stato ancora valutato o non è disponibile alcuna informazione sulla sua popolazione o sulle minacce che affronta.

Near Threatened (NT): Indica che una specie non è ancora classificata come vulnerabile, ma potrebbe diventarlo in futuro se le minacce che affronta non vengono affrontate o se la popolazione continua a diminuire.

Critically Endangered (CR): Indica che una specie è in grave pericolo di estinzione in natura. Le specie classificate come criticamente in pericolo hanno una popolazione estremamente bassa e/o affrontano minacce estreme che mettono a rischio la loro sopravvivenza nel breve termine.

Varies: Indica che lo stato di conservazione di una specie varia, a seconda delle diverse popolazioni o sotto-specie che possono essere classificate in modi diversi.

Data Deficient (DD): Indica che non ci sono dati sufficienti per valutare lo stato di conservazione di una specie. Questo può essere dovuto alla mancanza di informazioni sulla popolazione, sulle minacce o sulla distribuzione della specie.

In [167]:
import pandas as pd
import numpy as np
import re
import networkx as nx
import inflect
from networkx.algorithms import cluster, community
import itertools

df = pd.DataFrame(pd.read_csv('datasetAnimali/Animal Dataset.csv'))
df['index'] = df.index
df

,Animal,Height (cm),Weight (kg),Color,Lifespan (years),Diet,Habitat,Predators,Average Speed (km/h),Countries Found,Conservation Status,Family,Gestation Period (days),Top Speed (km/h),Social Structure,Offspring per Birth,index
0,Aardvark,105-130,40-65,Grey,20-30,Insectivore,"Savannas, Grasslands","Lions, Hyenas",40,Africa,Least Concern,Orycteropodidae,210-240,40,Solitary,1,0
1,Aardwolf,40-50,8-14,Yellow-brown,10-12,Insectivore,"Grasslands, Savannas","Lions, Leopards",24-30,Eastern and Southern Africa,Least Concern,Hyaenidae,90,40,Solitary,2-5,1
2,African Elephant,270-310,2700-6000,Grey,60-70,Herbivore,"Savannah, Forest","Lions, Hyenas",25,Africa,Vulnerable,Elephantidae,640-660,40,Herd-based,1,2
3,African Lion,80-110,120-250,Tan,10-14,Carnivore,"Grasslands, Savannas","Hyenas, Crocodiles",58,Africa,Vulnerable,Felidae,98-105,80,Group-based,2-4 (usually),3
4,African Wild Dog,75-80,18-36,Multicolored,10-12,Carnivore,Savannahs,"Lions, Hyenas",56,Sub-Saharan Africa,Endangered,Canidae,70,56,Group-based,10-12,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,Yak,140-160,500-1200,"Brown, Black",20-25,Herbivore,Mountains,"Snow Leopards, Wolves",24,"Himalayas, Central Asia",Least Concern,Bovidae,215-280,24,Group-based,10-50,200
201,Yellow-Eyed Penguin,60-65,1-3,"Yellow, White",Up to 20,Carnivore,Coastal Areas,"Seals, Orcas",25,New Zealand,Endangered,Spheniscidae,80-90,25,Solitary,1,201
202,Yeti Crab,Up to 15,Up to 0.5,"White, Hairy",Up to 20,Omnivore,Hydrothermal Vents,Not Applicable,Not Applicable,Pacific Ocean,Not Evaluated,Kiwaidae,Not Applicable,Not Applicable,Solitary,Not Applicable,202
203,Zebra,220-340,400-900,"Black, White",20-25,Herbivore,Grasslands,"Lions, Hyenas",25,Africa,Least Concern,Equidae,180-365,25,Group-based,5-20,203


In [168]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [169]:
def uniqueValues(column_name):
    return df[column_name].unique()

def contaValoriUnici(dataset,colonna):
    for i in dataset[colonna].unique():
        print("Valore ",i,"counts: ",(dataset[colonna] ==i).sum())

def addPredators(nome_preda, arr_predators):
    for nome_predatore in arr_predators:
        df.loc[df['Animal'] == nome_preda, 'Predators'] = df.loc[df['Animal'] == nome_preda, 'Predators'] + ', ' + nome_predatore
        
def replacePreators(nome_preda, arr_predators):
    df.loc[df['Animal'] == nome_preda, 'Predators'] = arr_predators[0]
    for nome_predatore in arr_predators:
        if nome_predatore != arr_predators[0]:
            df.loc[df['Animal'] == nome_preda, 'Predators'] = df.loc[df['Animal'] == nome_preda, 'Predators'] + ', ' + nome_predatore

def plural_to_singular(plural_words):
    p = inflect.engine()
    singular_words = []
    for word in plural_words:
        singular_word = p.singular_noun(word)
        if singular_word:
            singular_words.append(singular_word)
        else:
            singular_words.append(word)
    return singular_words

def plural_to_singular_word(word):
    p = inflect.engine()
    singular = p.singular_noun(word)
    if singular:
        return singular
    else:
        return word
    
#stampa archi entranti e uscenti di un certo animale
def visualize_node_edges(G, node_name):
    if node_name in G.nodes():
        successors = list(G.successors(node_name))
        predecessors = list(G.predecessors(node_name))
        
        print(f"Gli archi uscenti dal nodo {node_name} sono:")
        for successor in successors:
            print(f"{node_name} -> {successor}")
        
        print(f"\nGli archi entranti nel nodo {node_name} sono:")
        for predecessor in predecessors:
            print(f"{predecessor} -> {node_name}")
    else:
        print(f"Il nodo {node_name} non esiste nel grafo.")

def transform_weight(weight_str):
    if "Up to" in weight_str:
        return float(re.findall(r'\d*\.?\d+', weight_str)[0])
    else:
        if weight_str == 'Varies':
            return -1
        min_weight, max_weight = map(float, re.findall(r'\d*\.?\d+', weight_str))
        return (min_weight + max_weight) / 2
    
def label_weight(weight):
    if weight == -1:
        return 'Not Applicable'
    elif weight < 30:
        return 'Small'
    elif 30 <= weight <= 60:
        return 'Medium'
    else:
        return 'Heavy'


In [170]:
#stampa i nodi che contengono una certa parola
def find_nodes_containing_keyword(G, keyword):
    matching_nodes = []
    for node in G.nodes():
        if keyword.lower() in node.lower():
            matching_nodes.append(node)
    return matching_nodes

# keyword = "orangutan"
# matching_nodes = find_nodes_containing_keyword(G, keyword)
# if matching_nodes:
#     print(f"I nodi che contengono la parola '{keyword}' nel loro nome sono:")
#     for node in matching_nodes:
#         print(node)
# else:
#     print(f"Nessun nodo trovato contenente la parola '{keyword}'.")


## Droppare animali estinti, i predatori Not Applicable e quelli che non hanno una area geografica

In [171]:
# Elimino gli animali duplicati nella colonna Animal
indici_righe_da_eliminare = [82, 85, 106, 110, 124, 131]
df.drop(indici_righe_da_eliminare, inplace=True)

In [172]:
addPredators('African Elephant', ['Crocodile'])
replacePreators('Alpine Ibex', ['Mountain Tigers','Wolves', 'Lynxes', 'Bears', 'Foxes'])
replacePreators('Amazon Rainforest Frog', ['leimadophis epinephelus'])
replacePreators('Arabian Oryx', ['Striped Hyenas', 'Arabian Wolves', 'Lions'])
replacePreators('Arctic Fox', ['Polar Bears', 'Wolves', 'Golden Eagles', 'Grizzly Bears', 'Bald Eagles'])
replacePreators('Asian Elephant', ['Tigres'])
replacePreators('Atlantic Puffin', ['Great Skua', 'Great Black-backed Gull', 'Foxes'])
addPredators('Aye-Aye', ['Fossas'])
addPredators('Baird\'s Tapir', ['Pumas'])
replacePreators('Bald Eagle', ['Owls', 'Raccoons', 'Wolverines', 'Crows', 'Hawks'])
addPredators('Banded Palm Civet', ['Owls', 'Snakes'])
addPredators('Barbary Macaque', ['Domestic Dogs'])
addPredators('Bearded Dragon', ['Foxes'])
replacePreators('Blue Jay', ['Hawks', 'Owls', 'Falcons', 'Snakes'])
addPredators('Blue-Footed Booby	', ['Galapagos Hawk'])
replacePreators('Bornean Orangutan', ['Leopards', 'Bearded Pigs', 'Crocodiles', 'Pythons', 'Black Eagles'])
replacePreators('Bottlenose Dolphin', ['Tiger Sharks', 'Dusky Sharks', 'Bull Sharks', 'Great White Sharks'])
replacePreators('Brown Bear', ['Wolves', 'Cougars'])
replacePreators('Burmese Python', ['Asian Tigres', 'Leopards'])
addPredators('Capybara', ['Caimans'])
addPredators('Cheetah', ['Leopards'])
replacePreators('Chinese Giant Salamander', ['Otters', 'Foxes', 'Weasels', 'Badgers'])
replacePreators('Common Snapping Turtle', ['Foxes', 'Coyotes', 'Skunks', 'Minks', 'Fishers', 'Raccoons', 'Crows', 'Herons', 'Hawks', 'Owls', 'Bullfrogs', 'Fish', 'Snakes'])
replacePreators('Dhole', ['Not Applicable'])
replacePreators('Dingo', ['Crocodiles', 'Canid Species'])
addPredators('Dugong', ['Saltwater Crocodiles'])
addPredators('Eastern Gorilla', ['Crocodiles'])
addPredators('Echidna', ['Feral Cats', 'Foxes', 'Goannas'])
addPredators('Emperor Penguin', ['Sea Lions', 'Orcas'])
addPredators('Emperor Tamarin', ['Wild Cats', 'Dogs', 'Snakes', 'Birds of Prey'])
addPredators('Fennec Fox', ['Large Mammals'])
replacePreators('Flying Fox', ['Eagles', 'Owls', 'Pythons'])
addPredators('Galápagos Penguin', ['Sea Lions', 'Snakes', 'Owls', 'Hawks'])
replacePreators('Galápagos Tortoise', ['Galápagos hawk'])
addPredators('Gaur', ['Dhole', 'Saltwater Crocodiles'])
addPredators('Gerenuk', ['African Wild Dogs', 'Cheetahs', 'Hyenas'])
addPredators('Giant Panda', ['Yellow-Throated Martens', 'Eagles', 'Feral Dogs', 'Asian Black Bear'])
addPredators('Gila Monster', ['Hawks', 'Owls', 'Snakes'])
addPredators('Golden Lion Tamarin', ['Wild Cats'])
addPredators('Green Anaconda', ['Crab-eating Foxes', 'Tegu Lizards', 'Crested Caracaras'])
addPredators('Grevy\'s Zebra', ['Leopards', 'Hunting Dogs', 'Cheetah'])
addPredators('Harp Seal', ['Large Sharks'])
replacePreators('Hummingbird', ['Owls', 'Grackles', 'Blue Jays', 'Herons', 'Tanagers', 'Loggerhead Shrikes', 'Gulls'])
replacePreators('Japanese Macaque', ['Mountain Hawk Eagles', 'Feral Dogs', 'Raccoon Dogs'])
replacePreators('Kangaroo Rat', ['Swift Foxes', 'Badgers', 'Bobcats',  'Coyotes', 'Snakes', 'Rattlesnakes', 'Raptors', 'Owls'])
replacePreators('King Cobra', ['Birds', 'Mongooses'])
addPredators('Lemur', ['Boas'])
addPredators('Lion-tailed Macaque', ['Snakes', 'Raptors'])
addPredators('Lyrebird', ['Quolls', 'Dogs', 'Feral Cats', 'Foxes'])
addPredators('Mandrill', ['African Rock Pythons', 'Crowned Eagles'])
replacePreators('Markhor', ['Eurasian Lynxs', 'Snow Leopards','Himalayan Wolfes', 'Brown Beas'])
addPredators('Meerkat', ['Snakes'])
replacePreators('Okapi', ['Leopards', 'Servals', 'Golden Cats'])
addPredators('Orangutan', ['Large Pythons'])
addPredators('Pangolin', ['Leopards', 'Hyenas', 'Pythons'])
addPredators('Patagonian Mara', ['Felids', 'Grisons'])
replacePreators('Pink Fairy Armadillo', ['Foxes', 'Birds of Prey', 'Birds', 'Canids'])
replacePreators('Platypus', ['Snakes', 'Water Rats', 'Goannas', 'Foxes', 'Cats', 'Dogs'])
replacePreators('Praying Mantis', ['Frogs', 'Lizards', 'Spiders', 'Hornets', 'Ants', 'Birds', 'Bats'])
addPredators('Proboscis Monkey', ['Crocodiles', 'Pythons'])
replacePreators('Pronghorn', ['Wolves', 'Foxes', 'Coyotes', 'Bobcats', 'Golden Eagles'])
replacePreators('Rottweiler', ['Not Applicable'])
addPredators('Serval', ['Wild Dogs'])
replacePreators('Shoebill', ['Crocodiles'])
replacePreators('Siberian Husky', ['Not Applicable'])
addPredators('Sloth', ['Ocelots'])
addPredators('Slow Loris', ['Orangutans', 'Hawk Eagles'])
replacePreators('Snow Leopard', ['Humans'])
replacePreators('Spider Monkey', ['Jaguars', 'Eagles', 'Snakes'])
replacePreators('Spotted Hyena', ['Lions'])
replacePreators('Star-Nosed Mole', ['Raptors', 'Screech', 'Great Horned', 'Long-Eared', 'Barred', 'Barn Owls', 'Red-Tailed Hawks'])
addPredators('Sumatran Orangutan', ['Clouded Leopards', 'Large Pythons', 'Crocodiles'])
addPredators('Tarsier', ['Monitor Lizards', 'Raptors'])
replacePreators('Termite', ['Echidnas', 'Foxes', 'Galagos', 'Numbats', 'Mongooses', 'Mice', 'Pangolins', 'Genets', 'Civits', 'Bats', 'Echidnas', 'Moles', 'Shrews', 'Bibbies'])
addPredators('Three-Toed Sloth', ['Ocelots', 'Jaguars'])
replacePreators('Toco Toucan', ['Forest Eagles', 'Hawks', 'Owls', 'Boas', 'Jaguars', 'Margays'])
addPredators('Tuatara', ['Birds of Pray'])
replacePreators('Tufted Puffin', ['Bald Eagles', 'Peregrine Falcons', 'Snowy Owls', 'Eagle Owls', 'Sharks', 'Artic foxes', 'Red Foxes'])
addPredators('Uakari', ['Birds of Pray'])
addPredators('Vampire Bat', ['Eagles'])
addPredators('Vaquita', ['Orcas'])
replacePreators('Vulture', ['Hawks', 'Snakes', 'Wild Cats'])
addPredators('Warthog', ['Spotted Hyenas', 'Cheetahs', 'African Wild Dogs'])
addPredators('Water Buffalo', ['Lions'])
replacePreators('Wild Boar', ['Wolves'])
addPredators('Wildebeest', ['Hyenas', 'Cheetahs', 'Wild Dogs'])
addPredators('Wolverine', ['Mountain Lions'])
replacePreators('Wombat', ['Foxes', 'Dingoes', 'Wild Dogs', 'Eagles', 'Tasmanian Devils'])
addPredators('Woodpecker', ['Feral Cats', 'Foxes', 'Hawks', 'Coyotes'])
addPredators('Yellow-Eyed Penguin', ['Cats', 'Stoats', 'Dogs', 'Ferrets', 'Barracoota', 'Sharks', 'Sea Lions'])
addPredators('Zebra', ['Wild dogs', 'Cheetahs'])

In [173]:
df.Predators.unique()
df = df.rename(columns={'Predators': 'AllPredators'})

In [174]:
# Droppo animali estinti
df = df.loc[(df['Conservation Status'] != 'Extinct (around 4,000 years ago)') &
             (df['Conservation Status'] != 'Extinct (around 58 million years ago)') &
               (df['Conservation Status'] != 'Extinct')]

# Deroppo gli animali che non hanno predatori e che non hanno uno stato di conservazione definito
df = df.loc[(df['AllPredators'] != 'Not Applicable') & (df['Conservation Status'] != 'Not Applicable')]

In [175]:
# Estraiamo gli animali e i predatori in un unico elenco
animals = df['Animal'].tolist()
predators = df['AllPredators'].str.split(', ').explode().unique().tolist()

In [176]:
animali_duplicati = df[df.duplicated(subset=['Animal'], keep=False)]
animali_duplicati

,Animal,Height (cm),Weight (kg),Color,Lifespan (years),Diet,Habitat,AllPredators,Average Speed (km/h),Countries Found,Conservation Status,Family,Gestation Period (days),Top Speed (km/h),Social Structure,Offspring per Birth,index


In [177]:
sing_predators = plural_to_singular(predators)

animali_comuni = set(sing_predators).intersection(set(animals))

In [178]:
G = nx.DiGraph()

# Aggiungere nodi al grafo con lo stato di conservazione, se presente
for animal in animals:
    status = df[df['Animal'] == animal]['Conservation Status'].iloc[0]
    height = df[df['Animal'] == animal]['Height (cm)'].iloc[0]
    weight = df[df['Animal'] == animal]['Weight (kg)'].iloc[0]
    lifespan = df[df['Animal'] == animal]['Lifespan (years)'].iloc[0]
    habitat = df[df['Animal'] == animal]['Habitat'].iloc[0]
    family = df[df['Animal'] == animal]['Family'].iloc[0]
    gestation_period = df[df['Animal'] == animal]['Gestation Period (days)'].iloc[0]
    offspring_per_birth = df[df['Animal'] == animal]['Offspring per Birth'].iloc[0]
    countries = df[df['Animal'] == animal]['Countries Found'].iloc[0]
    pray = 'Yes'
    if animal in animali_comuni:
        pray = 'Not Only'
    G.add_node(animal, conservation_status = status, pray = pray, height = height, weight = weight, lifespan = lifespan, habitat = habitat,
               family = family, gestation_period = gestation_period, offspring_per_birth = offspring_per_birth, countries = countries)

# Aggiungere nodi predatori al grafo solo se non sono presenti in animali_comuni
for predator in sing_predators:
    if predator not in animali_comuni:
        G.add_node(predator, conservation_status = 'Not Have', pray = 'No')

for _, row in df.iterrows():
    animal = row['Animal']

    for predator in row['AllPredators'].split(', '):
        sing_predator = plural_to_singular_word(predator)
        G.add_edge(sing_predator, animal)


In [179]:
df['Weight (kg)'] = df['Weight (kg)'].str.replace(',', '.')
df['Weight (kg)'] = df['Weight (kg)'].apply(transform_weight)

#Adesso voglio raggruppare gli animali in leggeri, medi, grandi
df['weight_labeled'] = df['Weight (kg)'].apply(label_weight)

## ANALISI STRUTTURA DELLA RETE

Betweenness centrality: Misura il numero di volte che un nodo si trova sul percorso più breve tra tutte le coppie di nodi nella rete. I nodi con betweenness elevato possono essere considerati "ponti" importanti tra le parti della rete --> utile per trovare gli animali che sono sia prede che predatori più rilevanti nel nostro dataset e che qundi potrebbero essere complici di un eventuale danno a catena indiretto

In [180]:
# Betweenness Centrality --> più il valore è alto più il nodo 
betweenness_centrality = nx.betweenness_centrality(G)

print("Betweenness Centrality:", betweenness_centrality)
max_bc_node = max(betweenness_centrality, key=betweenness_centrality.get)
max_bc_value = betweenness_centrality[max_bc_node]

Betweenness Centrality: {'Aardvark': 0.0, 'Aardwolf': 0.0, 'African Elephant': 0.0, 'African Lion': 0.0, 'African Wild Dog': 4.807414956829413e-06, 'Alpine Ibex': 0.0, 'Amazon Rainforest Frog': 0.0, 'American Bison': 0.0, 'Anteater': 0.0, 'Arabian Oryx': 0.0, 'Arctic Fox': 0.0, 'Arowana': 0.0, 'Asian Elephant': 0.0, 'Atlantic Puffin': 0.0, 'Atlantic Spotted Dolphin': 0.0, 'Australian Fur Seal': 0.0, 'Axolotl': 0.0, 'Aye-Aye': 0.0, "Baird's Tapir": 0.0, 'Bald Eagle': 0.00011858290226845886, 'Banded Palm Civet': 0.0, 'Barbary Macaque': 0.0, 'Basking Shark': 0.0, 'Bearded Dragon': 0.0, 'Bengal Fox': 0.0, 'Bengal Tiger': 0.0, 'Black Rhinoceros': 0.0, 'Blue Jay': 2.8844489740976482e-05, 'Blue Morpho Butterfly': 0.0, 'Blue Whale': 0.0, 'Blue-Footed Booby': 0.0, 'Blue-Ringed Octopus': 0.0, 'Bonobo': 0.0, 'Bornean Orangutan': 0.0, 'Bottlenose Dolphin': 0.0, 'Brazilian Wandering Spider': 0.0, 'Brown Bear': 0.0, 'Burmese Python': 0.0, 'Capybara': 0.0, 'Cheetah': 2.4037074784147066e-05, 'Chimpanz

In [181]:
# Trova i 10 nodi con i valori più alti di Betweenness Centrality
top_10_bc_nodes = sorted(betweenness_centrality, key=betweenness_centrality.get, reverse=True)[:10]

# Stampa i risultati
print("I 10 nodi con i valori più alti di Betweenness Centrality:")
for node in top_10_bc_nodes:
    print(node, ":", betweenness_centrality[node])


I 10 nodi con i valori più alti di Betweenness Centrality:
Wolf : 0.00021312872975277065
Tiger : 0.00013460761879122358
Bald Eagle : 0.00011858290226845886
Dingo : 9.614829913658826e-05
Fossa : 7.691863930927061e-05
Wolverine : 6.08939227865059e-05
Orangutan : 3.845931965463531e-05
Snow Leopard : 3.3651904697805894e-05
Blue Jay : 2.8844489740976482e-05
Cheetah : 2.4037074784147066e-05


Il coefficiente di clustering di un grafo misura il grado in cui i nodi nel grafo tendono a formare cluster o raggruppamenti.

In [182]:
# Calcolo del clustering coefficient
clustering_coefficient = nx.average_clustering(G)
print("Clustering Coefficient:", clustering_coefficient)

Clustering Coefficient: 0.025143159658844356


Quando esegui il rilevamento delle comunità su questo grafo, l'algoritmo cercherà di identificare gruppi di animali che sono più fortemente connessi tra loro rispetto alle connessioni con il resto del grafo --> utile per ottenre dei sotto-gruppi fortemente relazionati tra di loro, il che potrebbe anche implicare animali che appartengono allo stesso habitat

In [183]:
# Rilevamento delle comunità utilizzando l'approccio della modularity maximization
communities = community.greedy_modularity_communities(G)

# Creazione di un dizionario per mappare ogni specie alla sua comunità
community_mapping = {}
for idx, com in enumerate(communities):
    for species in com:
        community_mapping[species] = idx

# Stampare le comunità identificate
print("Community Detection:")
for species, community_id in community_mapping.items():
    print(species, ":", community_id)


Community Detection:
Bengal Tiger : 0
Gharial : 0
Iberian Lynx : 0
Human : 0
Serval : 0
Mountain Gorilla : 0
Proboscis Monkey : 0
Larger Predator : 0
Wildebeest : 0
Grevy's Zebra : 0
Crowned Eagle : 0
African Rock Python : 0
Cheetah : 0
Spotted Hyena : 0
Fossa : 0
Tiger : 0
Sumatran Orangutan : 0
Western Gorilla : 0
African Lion : 0
Sumatran Rhinoceros : 0
Clouded Leopard : 0
Chimpanzee : 0
Warthog : 0
Coquerel's Sifaka : 0
Sun Bear : 0
Black Rhinoceros : 0
Burmese Python : 0
Lion : 0
Wildcat : 0
Golden Cat : 0
African Elephant : 0
Saola : 0
Komodo Dragon : 0
Wild dog : 0
Mandrill : 0
White Tiger : 0
Orangutan : 0
Eastern Gorilla : 0
Pangolin : 0
Large Python : 0
Zebra : 0
Bonobo : 0
Okapi : 0
Sumatran Tiger : 0
Naked Mole Rat : 0
Aardwolf : 0
Leopard : 0
African Wild Dog : 0
Hippopotamus : 0
White Rhinoceros : 0
Asian Tigre : 0
Gerenuk : 0
Hyena : 0
Sumatran Rhino : 0
Water Buffalo : 0
Crocodile : 0
Aye-Aye : 0
Shoebill : 0
Western Lowland Gorilla : 0
Wild Dog : 0
Hunting Dog : 0
Indr

Identifico le sottoreti presenti nella mia rete globale --> Utile per identificare animali chiave con pochi predaotori

In [204]:
# Identificazione di sottoreti dense
weakly_connected_components = list(nx.weakly_connected_components(G))

# Creazione di sottografi per ciascuna componente connessa debole
subgraphs = [G.subgraph(component) for component in weakly_connected_components]

# Stampare le dimensioni delle sottoreti dense
print("Dimensioni delle sottoreti dense:")
for i, subgraph in enumerate(subgraphs):
    print("Sottorete", i+1, ":", subgraph)


Dimensioni delle sottoreti dense:
Sottorete 1 : DiGraph with 312 nodes and 499 edges
Sottorete 2 : DiGraph with 2 nodes and 1 edges
Sottorete 3 : DiGraph with 2 nodes and 1 edges
Sottorete 4 : DiGraph with 2 nodes and 1 edges
Sottorete 5 : DiGraph with 2 nodes and 1 edges
Sottorete 6 : DiGraph with 2 nodes and 1 edges
Sottorete 7 : DiGraph with 2 nodes and 1 edges


In [205]:
def getSubNetNodes(sub_net_nodes):
    # Lista per memorizzare i nodi delle sottoreti dalla 2 alla 7
    nodes_in_subnet = []
    i = sub_net_nodes - 1
    # Itera attraverso i sottografi dalla 2 alla 7
    for subgraph in subgraphs[i:sub_net_nodes]:  # Partendo dall'indice 1 per selezionare il secondo sottografo
        # Aggiungi i nodi del sottografo corrente alla lista
        nodes_in_subnet.extend(subgraph.nodes())

    # Rimuovi i duplicati, se presenti
    nodes_in_subnet = list(set(nodes_in_subnet))

    # Stampa i nodi delle sottoreti dalla 2 alla 7
    print("Nodi nella sottorete:", nodes_in_subnet)

    #return nodes_in_subnet


In [206]:
getSubNetNodes(2)
getSubNetNodes(3)
getSubNetNodes(4)
getSubNetNodes(5)
getSubNetNodes(6)
getSubNetNodes(7)

Nodi nella sottorete: ['leimadophis epinephelu', 'Amazon Rainforest Frog']
Nodi nella sottorete: ['Tigre', 'Asian Elephant']
Nodi nella sottorete: ['Galápagos Tortoise', 'Galápagos hawk']
Nodi nella sottorete: ['Galliformes', 'Vary']
Nodi nella sottorete: ['Hagfish', 'Various Predator']
Nodi nella sottorete: ['Shortfin Mako Shark', 'Larger Shark']


In [207]:
# Rilevamento delle comunità utilizzando l'approccio della modularity maximization
communities = community.greedy_modularity_communities(G)

# Calcolo della betweenness centrality
betweenness = nx.betweenness_centrality(G)

# Creazione di un dizionario per mappare ogni specie alla sua comunità e alla sua betweenness centrality
node_attributes = {}
for idx, com in enumerate(communities):
    for species in com:
        node_attributes[species] = {'community': idx, 'betweenness_centrality': betweenness[species]}

# Aggiunta dei valori delle comunità e della betweenness centrality come attributi ai nodi del grafo
nx.set_node_attributes(G, node_attributes)

# Stampare le informazioni identificate per ogni nodo
print("Node Information:")
for node, data in G.nodes(data=True):
    print("Node:", node, "Community:", data['community'], "Betweenness Centrality:", data['betweenness_centrality'])


Node Information:
Node: Aardvark Community: 0 Betweenness Centrality: 0.0
Node: Aardwolf Community: 0 Betweenness Centrality: 0.0
Node: African Elephant Community: 0 Betweenness Centrality: 0.0
Node: African Lion Community: 0 Betweenness Centrality: 0.0
Node: African Wild Dog Community: 0 Betweenness Centrality: 4.807414956829413e-06
Node: Alpine Ibex Community: 1 Betweenness Centrality: 0.0
Node: Amazon Rainforest Frog Community: 14 Betweenness Centrality: 0.0
Node: American Bison Community: 1 Betweenness Centrality: 0.0
Node: Anteater Community: 4 Betweenness Centrality: 0.0
Node: Arabian Oryx Community: 12 Betweenness Centrality: 0.0
Node: Arctic Fox Community: 1 Betweenness Centrality: 0.0
Node: Arowana Community: 9 Betweenness Centrality: 0.0
Node: Asian Elephant Community: 15 Betweenness Centrality: 0.0
Node: Atlantic Puffin Community: 1 Betweenness Centrality: 0.0
Node: Atlantic Spotted Dolphin Community: 3 Betweenness Centrality: 0.0
Node: Australian Fur Seal Community: 3 Betwe

In [208]:
df.columns

Index(['Animal', 'Height (cm)', 'Weight (kg)', 'Color', 'Lifespan (years)',
       'Diet', 'Habitat', 'AllPredators', 'Average Speed (km/h)',
       'Countries Found', 'Conservation Status', 'Family',
       'Gestation Period (days)', 'Top Speed (km/h)', 'Social Structure',
       'Offspring per Birth', 'index', 'weight_labeled'],
      dtype='object')

In [221]:
def jaccard_similarity_by_community(nome_feature):
    # Dizionario per mappare ogni comunità ai suoi stati di conservazione
    community_conservation_status = {}

    # Riempire il dizionario community_conservation_status con gli stati di conservazione di ogni comunità
    for species, community_id in community_mapping.items():
        conservation_status = df.loc[df['Animal'] == species, nome_feature].values
        print(conservation_status)
        if len(conservation_status) > 0:
            conservation_status = conservation_status[0]
            if community_id not in community_conservation_status:
                community_conservation_status[community_id] = set()
            community_conservation_status[community_id].add(conservation_status)

    similarities = []

    # Calcoliamo la Jaccard Similarity tra tutte le coppie di comunità
    for pair in itertools.combinations(community_conservation_status.keys(), 2):
        conservation_status_set1 = community_conservation_status[pair[0]]
        conservation_status_set2 = community_conservation_status[pair[1]]
        
        intersection = len(conservation_status_set1.intersection(conservation_status_set2))
        union = len(conservation_status_set1.union(conservation_status_set2))
        jaccard_similarity_score = intersection / union if union != 0 else 0
        
        similarities.append((pair, jaccard_similarity_score))

    # Ordiniamo le coppie in base al grado di similarità
    similarities.sort(key=lambda x: x[1], reverse=True)

    return similarities

In [222]:
#Jaccard Similarity tra comunità in base allo stato di conservazione
#posso applicare la jaccard similarity anche in base all'habitat, dieta, e peso degli animali per comunità

similarities = jaccard_similarity_by_community('Conservation Status')

# Stampa le prime 10 coppie con il grado di similarità maggiore
for i in range(min(10, len(similarities))):
    pair, similarity = similarities[i]
    print(f"Community {pair[0]} e Community {pair[1]} con similarità {similarity}")


['Endangered']


TypeError: 'int' object is not callable

Connectance: rappresenta la percentuale di connessioni effettivamente presenti rispetto al numero massimo di connessioni possibili.

Edge-to-Node-Ratio: fornisce una stima del numero medio di collegamenti (archi) per nodo nella rete.

In [ ]:
#Connectance & Edge-to-Node-Ratio

# Calcolo del numero totale di nodi e di archi nel grafo
total_nodes = G.number_of_nodes()
total_edges = G.number_of_edges()

print("Numero totale di nodi nel grafo:", total_nodes)
print("Numero totale di archi nel grafo:", total_edges)

# Calcolo della Connectance
connectance = total_edges / (total_nodes ** 2)

# Calcolo dell'Edge-to-Node-Ratio
edge_to_node_ratio = total_edges / total_nodes

print("Connectance:", connectance)
print("Edge-to-Node-Ratio:", edge_to_node_ratio)


In [ ]:
#nx.write_gexf(G, 'new_animals_cleared.gexf')